In [ ]:
# Standard modules needed
import numpy as np
import pandas as pd
import datetime as dt
from types import SimpleNamespace
from scipy import optimize
from scipy import interpolate
%load_ext autoreload
%autoreload 2

# Plotting tools
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.ticker

# User written modules
import examq1 as q1
import examq2 as q2

# Windmill industry in Denmark

In [ ]:
import requests

url = "https://ens.dk/sites/ens.dk/files/Statistik/anlaeg.xlsx"
r = requests.get(url)

with open('windmills.xlsx', 'wb') as xls_file:
    xls_file.write(r.content)   

## Question 1

#### 1. Loading and cleaning the data
To load the data, I use the user written module examq1.

In [ ]:
# Load data and make final adjustments
nondec_df = q1.load_windmill_data('IkkeAfmeldte-Existing turbines')
nondec_df['decom'] = 0                  # dummy for being decommissioned
nondec_df = nondec_df.iloc[:-3,:]       # drop column totals

dec_df = q1.load_windmill_data('Afmeldte-Decommissioned')
dec_df['decom'] = 1                                         # dummy for being decommissioned
dec_df = dec_df.iloc[:-1,:]                                 # drop column totals
dec_df['decom_date'] = pd.to_datetime(dec_df.decom_date)    # format decommission date

# Concatenate to final dataframe
windmill_df = pd.concat([nondec_df, dec_df], join='inner')

#### 2. Plot total electricity production 1977-2021

In [ ]:
# Calculate year-by-year total production
years = []          # list of years
prod_total = []     # list of total production

for col in windmill_df.columns:
    if 'year' in col:
        years.append(int(col[-4:]))
        prod_total.append(windmill_df[col].sum()/1_000_000)

# Convert to DataFrame 
windmill_yearly = pd.DataFrame({'year': years, 'prod_totalGWh': prod_total})
windmill_yearly.set_index('year', inplace=True)

In [ ]:
# Plot!
fig, ax = plt.subplots(figsize=(10,6))
ax.plot(windmill_yearly.index, windmill_yearly.prod_totalGWh)

ax.set_ylabel("GWh")
ax.set_title("Total energy production from windmills")
ax.get_yaxis().set_major_formatter(
    matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
ax.grid(True);

## Question 2

#### 1. Average and maximum turbine capacity

In [ ]:
# Variable for connection year
windmill_df['connect_y'] = pd.DatetimeIndex(windmill_df.connect_date).year

# Calculate capacity by connection year and add to yearly dataframe
for year in years:
    mean = windmill_df[windmill_df.connect_y == year].capkW.mean()
    max = windmill_df[windmill_df.connect_y == year].capkW.max()

    windmill_yearly.loc[year, 'mean_capacitykWh']= mean*(365.25*24)/10_000
    windmill_yearly.loc[year, 'max_capacitykWh'] = max*(365.25*24)/10_000

In [ ]:
# Plot!
fig, ax = plt.subplots(1,1, figsize = (10,6), sharey=True)

ax.plot(windmill_yearly.index, windmill_yearly.mean_capacitykWh, label='Mean capacity')
ax.plot(windmill_yearly.index, windmill_yearly.max_capacitykWh, label='Maximum capacity')

ax.set_ylabel('10.000 kWh')
ax.set_xlabel('Date of original connection to grid')
ax.set_title('Windmill energy production capacity')
ax.get_yaxis().set_major_formatter(
    matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
ax.legend()
ax.grid();


#### 2. 7 year moving average of average and maximum capacity

In [ ]:
# Calculate 7 year moving average
q1.MA7(windmill_yearly, 'mean_capacitykWh')
q1.MA7(windmill_yearly, 'max_capacitykWh')

In [ ]:
# Plot!
fig, ax = plt.subplots(1,1, figsize = (10,6), sharey=True)

ax.plot(windmill_yearly.index, windmill_yearly.MAmean_capacitykWh, label='Mean capacity, 7 year moving average')
ax.plot(windmill_yearly.index, windmill_yearly.MAmax_capacitykWh, label='Maximum capacity, 7 year moving average')

ax.set_ylabel('10.000 kWh')
ax.set_xlabel('Date of original connection to grid')
ax.set_title('Windmill energy production capacity')
ax.get_yaxis().set_major_formatter(
    matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
ax.legend()
ax.grid();

#### 3. Potential capacity

In order to filter out decommisioned turbines, I merge the variable for decommission date from `dec_df` on to `windmill_df`.

In [ ]:
# join to decommision date from dec_df
windmill_df = windmill_df.merge(dec_df[['turbine_id','decom_date']], on='turbine_id', how='outer')

Next, I reshape the data to be able to aggregate over production years.

In [ ]:
# reshape to long form
year_vars = [var for var in windmill_df.columns if 'year' in var]
wm_long = pd.melt(windmill_df, id_vars='turbine_id', value_vars = year_vars)
wm_long['year'] = wm_long.variable.str[-4:].astype(int)
wm_long['productionkWh'] = wm_long.value
wm_long = wm_long[['year', 'turbine_id', 'productionkWh']]

# merge to data for capacity and active period
wm_long  = wm_long.merge(windmill_df[['turbine_id', 'connect_date', 'decom_date', 'capkW','loc_type']],on='turbine_id')

In order to adjust for years in whicch turbines are inactive or only active during part of the year, at define a variable `scale` equal to the share of the year, in which the turbine is active. I then calculated scaled capacity `capkWh_scale` as the windmill capacity in kWh weighted by `scale`.

In [ ]:
# scale down if activity is limited
wm_long['connect_year'] = pd.DatetimeIndex(wm_long.connect_date).year
wm_long['decom_year'] = pd.DatetimeIndex(wm_long.decom_date).year

wm_long['scale'] = 1
wm_long.loc[wm_long.year < wm_long.connect_year,'scale'] = 0 # no contribution before connection
wm_long.loc[wm_long.year > wm_long.decom_year,'scale'] = 0 # no contribution after decommission
wm_long.loc[wm_long.year == wm_long.connect_year, 'scale'] = (366 - wm_long.connect_date.dt.dayofyear)/365 # scale down if connected during year
wm_long.loc[wm_long.year == wm_long.decom_year, 'scale'] = wm_long.decom_date.dt.dayofyear/365 # scale down if disconnected during year
wm_long['capkWh_scale'] = wm_long.capkW*wm_long.scale*365.25*24

# Calculate total capacity and add to yearly dataframe
for year in years:
    totalcap = wm_long[wm_long.year == year].capkWh_scale.sum()
    windmill_yearly.loc[year, 'total_capacityGWh'] = totalcap/1_000_000 

In [ ]:
# Plot!
fig, ax = plt.subplots(figsize=(10,6))
ax.plot(windmill_yearly.index, windmill_yearly.prod_totalGWh, label='Actual energy production')
ax.set_ylabel("GWh")
ax.set_title("Total energy production from windmills")
ax.get_yaxis().set_major_formatter(
    matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
ax.plot(windmill_yearly.index, windmill_yearly.total_capacityGWh, label='Total production capacity')
ax.legend()
ax.grid();

From the figure it is evident that even though the total electricity production by windmills has increased drastically over the past 30 years, we are only producing less than a third of total production capacity. 

## Question 3

Before tackling the question, I drop observations which are presumably faulty or otherwise would blur the picture.

Approximately 400 windmills appearantly have a height of less than 1 meter. I assume these are mistakes and drop them from the data. I Also drop windmills that are decommissioned before or during 2021, windmills that produce no energy, and windmills with no data for energy.

#### 1. Height and productivity

In [ ]:
# Copy of windmill_df 
windmill_df3  = windmill_df.copy() 
windmill_df3.heightm = windmill_df3.heightm.astype(float)

# Drop very short windmills
windmill_df3 = windmill_df3[windmill_df3.heightm>1]

# Windmills decommissioned before 2021
decom = windmill_df3[pd.DatetimeIndex(windmill_df3.decom_date).year <= 2021]
windmill_df3.drop(decom.index, inplace=True)

# Windmills with no production
noprod = windmill_df3[windmill_df3.year2021 <= 0]
windmill_df3.drop(noprod.index, inplace = True)

# No data on energy production
nodata = windmill_df3[windmill_df3['year2021'].isnull()]
windmill_df3.drop(nodata.index, inplace=True)

To illustrate the relationship between windmill height and energy production, I use a simple OLS regression. A brief inspection of the data reveals that the relationship between the two is approximately log-linear (appart from a few shorter windmills). I estimate the model
$$
\begin{aligned}
\log(production_i) = \beta_0 + \beta_1\log(heightm_i) + \epsilon
\end{aligned}
$$

In [ ]:
lfit_total, betahat_total = q1.ols_predict(np.log(windmill_df3.year2021), np.log(windmill_df3.heightm))
print(f"Estimate of beta_1: {betahat_total[1]:4.2f}")

The estimate of $\beta_1$ is $\hat{\beta_i} = 3.95$, meaning that a height increase og 1 pct. is associated with an increase in yearly energy production of 3.95 pct. The relationship is ullistrated in the plot below.

In [ ]:
# Plot!
fig, ax = plt.subplots(figsize=(10,6))

ax.scatter(x=np.log(windmill_df3.heightm), y=np.log(windmill_df3.year2021), label='Observed energy production')
ax.plot(np.log(windmill_df3.heightm), lfit_total, label='Linear fit', color='C1')
ax.set_xlabel('Log height in meters')
ax.set_ylabel('Log energy production in kWh')
ax.legend(loc='lower right')
ax.set_title('Windmill height and energy production')
ax.grid();

#### Height and productivity, on-shore vs. off-shore windmills.

I continue to perform the same estimation while splitting the sample into on-shore and off-shore turbines. 

In [ ]:
# Format location type
windmill_df3.loc_type = windmill_df3.loc_type.str.lower()

# Estimate
lfit_land, betahat_land = q1.ols_predict(np.log(windmill_df3[windmill_df3.loc_type=='land'].year2021),
                                      np.log(windmill_df3[windmill_df3.loc_type=='land'].heightm))
lfit_hav, betahat_hav = q1.ols_predict(np.log(windmill_df3[windmill_df3.loc_type=='hav'].year2021), 
                                    np.log(windmill_df3[windmill_df3.loc_type=='hav'].heightm))


print(f"Estimated beta_1, on-shore turbines: {betahat_land[1]:5.2f}")
print(f"Estimated beta_1, off-shore turbines: {betahat_hav[1]:3.2f}")

In [ ]:
# Plot!
fig, (ax0, ax1) = plt.subplots(1,2, figsize=(20,6), sharey = True, sharex = True)

# On-shore turbines
ax0.scatter(x=np.log(windmill_df3[windmill_df3.loc_type=='land'].heightm), 
            y=np.log(windmill_df3[windmill_df3.loc_type=='land'].year2021), 
            label='Observed energy production')
ax0.plot(np.log(windmill_df3[windmill_df3.loc_type=='land'].heightm), 
        lfit_land, color='C1', label='Linear fit')

ax0.set_xlabel('Log height in meters')
ax0.set_ylabel('Log energy production in kWh')
ax0.legend(loc='lower right')
ax0.set_title('Windmill height and energy production, on-shore')
ax0.grid()

# Off-shore turbines
ax1.scatter(x=np.log(windmill_df3[windmill_df3.loc_type=='hav'].heightm), 
            y=np.log(windmill_df3[windmill_df3.loc_type=='hav'].year2021), 
                 label='Observed energy production')
ax1.plot(np.log(windmill_df3[windmill_df3.loc_type=='hav'].heightm), 
         lfit_hav, color='C1', label='Linear fit')

ax1.set_xlabel('Log height in meters')
ax1.set_ylabel('Log energy production in kWh')
ax1.legend(loc='lower right')
ax1.set_title('Windmill height and energy production, off-shore')
ax1.grid();


For both on-shore and off-shore, the relation ship between height and productivity is positive. A 1 percent height increase is associated with a production increase of 3.86 and 2.40 percent, respectively. 

Note that many of the off-shore turbines produce the exact same amount of energy. I assume this is because the production in larger off-shore windmill parks is assumed to be evenly split between turbines, but it could also indicate problems with the quality of the data.

#### 3. Capacity utilization

In order to calculate yearly average difference in production capacity and actual production, I return to the long form dataset. Once again, I drpo decommisioned and inactive windmills. Note thatI also drop mills in the year that they are decommissioned or connected, since I would expect them to have sub-capacity production in these years.

In [ ]:
# Copy dataframe and calculate capacity difference
wm_long3 = wm_long.copy()
wm_long3['capkWh'] = wm_long3.capkW*365*24
wm_long3['cap_diff'] = wm_long3.capkWh-wm_long3.productionkWh
wm_long3['cap_diff_pct'] = wm_long3.cap_diff/wm_long3.capkWh

# drop decommissioned
decom = wm_long3.year >= wm_long3.decom_year
wm_long3.drop(wm_long3[decom].index, inplace=True)

# drop inactive
inactive = wm_long3.year <= wm_long3.connect_year
wm_long3.drop(wm_long3[inactive].index, inplace=True)

# Format location type
wm_long3['loc_type'] = wm_long3.loc_type.str.lower()

# Calculate yearly average capacity difference and add to yearly dataframe
for loc in ['land', 'hav']:
    # aboslute
    col_name = 'meancapdiff_'+loc + 'kWh'
    for year in range(1990, 2022):
        meandiff = wm_long3[wm_long3.year == year][wm_long3.loc_type == loc].cap_diff.mean()
        windmill_yearly.loc[year, col_name] = meandiff

    # percent
    col_name = 'meancapdiff_'+loc+'pct'
    for year in range(1990, 2022):
        meandiff_pct = wm_long3[wm_long3.year == year][wm_long3.loc_type == loc].cap_diff_pct.mean()
        windmill_yearly.loc[year, col_name] = meandiff_pct

In [ ]:
# Plot!
fig, (ax0, ax1) = plt.subplots(1,2, figsize=(20,6))

# absolute difference
ax0.plot(windmill_yearly.index, windmill_yearly.meancapdiff_landkWh/10_000., label='On-shore windmills')
ax0.plot(windmill_yearly.index, windmill_yearly.meancapdiff_havkWh/10_000, label='Off-shore windmills')
ax0.set_ylabel('10.000 kWh')
ax0.legend()
ax0.get_yaxis().set_major_formatter(
    matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
ax0.set_title('Average difference between capacity and production')
ax0.grid()

# percentage difference
ax1.plot(windmill_yearly.index, windmill_yearly.meancapdiff_landpct*100, label='On-shore windmills')
ax1.plot(windmill_yearly.index, windmill_yearly.meancapdiff_havpct*100, label='Off-shore windmills')
ax1.set_ylabel('Percent')
ax1.legend()
ax1.set_title('Average difference between capacity and production, percent of total capacity')
ax1.grid();

While the average absolute difference between capcity and production has been rising for both on-shore and off-shore windmills, unused capacity for off-shore windmills increases dramatically more that of on-shore windmills, especially around 2000. However, this can reflect that the overall capacity for production is higher in off-shore windmills.

When examining the capacity difference as a percentage of total capacity, it becomes evident that this is actually the case. The relative unused capacity for production is actually higher for on-shore than for off-shore windmills. Moreover, the share of unused capacity in on-shore windmills has been constant since 1990, at 75-80 percent of total capacity. Meanwhile, the share of unused capacity in off-shore windmills dropped dramatically from 75 pct. in 2000 to 60 percent in 2010 and has en roughly constant since. 

# A discrete-continuous consumption-saving model

In [ ]:
# Parameters
par = SimpleNamespace()
par.rho = 8.0
par.nu = 0.1
par.kappa = 1
par.beta = 0.90
par.tau = 0.8
par.gamma = 1.2
par.ybar = 1.5
par.r = 0.04
par.p = 0.5
par.Delta = 0.4

m_min = par.tau+1e-5    # minimum value for m - must be possible to pay for studying
m_max = 5.0         # maximum value for m
grid_n = 100        # grid density


#### Question 1
To solve the model, I use the user written module examq2.

I solve the model by starting in period 2. Here, I solve the model for the optimal level of consumption in period 2, $c_2^*$ over a grid of cash on hand $m_2$. I store the results in  a vector of optimal consumption points, `c2_grid`. I also calculate the utility value of these consumption points and store the results in a vector `v2_grid`. I use `v2_grid` to set up a linear interpolater `v2_interp` for the value of cash on hand in period 2, which will be of use when solving period 1.

In [ ]:
# set up grid over m
m_grid = np.linspace(m_min, m_max, grid_n)

# solve period 2 over grid of m_2
v2_grid, c2_grid = np.array([q2.v2(m, par) for m in m_grid]).T

# set up interpolater
v2_interp = interpolate.RegularGridInterpolator([m_grid], v2_grid,
                                                bounds_error=False,fill_value=None)

Next, I solve for the optimal choice of consumption and whether or not to study in period 1 over a grid of cash on hand $m_1$. For a given $m_1$ I do this by solving the model once where the agent studies and once shere she doesn't. The optimal choice of study is the solution that yields the highest utility value and its corresponding sonsumption and study choice. I store the results in a vector of optimal consumption, `c1_grid` and a vector of optimal study choice `s_grid`. I store the corresponding utility values in a vector `v1_grid`.

In [ ]:
v1_grid, c1_grid, s_grid = np.array([q2.v1(m, par, v2_interp) for m in m_grid]).T

Finally, I plot the results.

In [ ]:
fig, (ax0, ax1, ax2) = plt.subplots(1,3, figsize=(24,6))

# value functions
ax0.plot(m_grid, v1_grid, label='$v_1(m_1)$')
ax0.plot(m_grid, v2_grid, label='$v_2(m_2)$')
ax0.set_xlabel('$m_1$/$m_2$')
ax0.set_ylabel('$v_1$/$v_2$')
ax0.set_title('Value of cash on hand')
ax0.legend()
ax0.grid()

# consumption functions
ax1.plot(m_grid, c1_grid, label='$c_1^*(m_1)$')
ax1.plot(m_grid, c2_grid, label='$c_2^*(m_2)$')
ax1.set_xlabel('$m_1$/$m_2$')
ax1.set_ylabel('$c_2$/$c_2$')
ax1.set_title('Consumption choice')
ax1.legend(loc='lower right')
ax1.grid()

# study choice
ax2.plot(m_grid, s_grid, label='$s^*$')
ax2.set_xlabel('$m_1$')
ax2.set_ylabel('s^*')
ax2.set_title('Study choice')
ax2.legend(loc='lower right')
ax2.grid();

Panel 1 of the above figure shows that the utility value $v_n$ is increasing in cash on hand $m_n$, which is to be expected. For a given level of $m_n$, the value at time 2 is higher than at time 1, $v_2(m)>v_1(m)$. This is because $m_1$ is supposed to finance consumption in two periods, where one is discounted with a rate of $\beta$, whereas $m_2$ only finances consumption that is not discounted.

Panel 2 shows the consumption profiles in periods 1 and 2. In period 1, the consumption profile is kinked at around $m_1\approx1.25$. This is due to the agent being credit constraint. For $m_1\leq1.25$, the agent consumes her entire cash on hand at time 1, and only for $m_1 > 1.25$ will she save betwwen periods.
The consumption profile at time 2 also has a kink, at approximately $m_2 \approx 1.3$. This kink is attributable to the bequest motive. For $m_2 > 1.3$ the agent will leave behind assets for her bequeathers in stead of consuming her entire cash on hand.

The consumption profile in period 1 has a discontinuity at around $m_1=2.28$. This is explained by panel 3 - the optimal studying profile. Panel 3 shows that $m_1 = 2.28$ is the point where it becomes optimal for the agent to reduce her consumption in favor of paying to study. Hereafter, there is a short inteval of $m_1$ where the credit constraint is binding, and the entir cash on hand is used on studying and consumption. At approximately $m_1 \approx 2.8$, it once again becomes optimal for the consumer to save, which explains the second kin in the consumption profile.

#### Question 2

Let $\tilde{\tau}_3$ denote the value of $\tau$ that makes a consumer with $m_1=3$ indifferent between studying nad not studying. Then, any $\tau < \tilde{\tau}_3$ must make it optimal for the consumer to study, while $\tau > \tilde{tau}_3$ makes it optimal not to study.

To find the value of $\tilde{\tau}_3$, I define the function `study_gain`, which for a given value of $\tau$ returns the utility value gain from studying. At $\tilde{\tau}_3$, `study_gain` must equal 0. Therefore, I can compute the value of $\tilde{\tau}_3$ by calling a root finder on `study_gain`.

In [ ]:
# Find root
obj = lambda tau: q2.study_gain(3, tau, v2_interp, par)
res  = optimize.root_scalar(obj, bracket=(0,3 - 1e-08)) # tau_tilde must be between 0 and full cash on hand value
assert res.converged

root = res.root
print(f'Consumer is indifferent at tau = {root:4.3f}')

So for any $\tau > 1.123$, it is not optimal for a consumer with $m_1=3$ to study. 

The plot below illustrates the result by plotting `study_gain` against different values of $\tau$. The function crosses the x-axis at exactly $\tau = 1.123$

In [ ]:
tau_grid = np.linspace(0, 1.5, grid_n)
study_gain_grid = [q2.study_gain(3, tau, v2_interp, par) for tau in tau_grid]

fig, ax = plt.subplots(figsize=(10,6))
ax.scatter(root, 0, zorder=2)
ax.plot(tau_grid, study_gain_grid, zorder = 1)
ax.axhline(0, color = 'black', alpha = 0.5, zorder=0)
ax.set_ylabel('Value gain')
ax.set_xlabel(r'$\tau$')
ax.set_title('Extra value from studying')
ax.grid();

# Approximating a function

#### Question 1
Below, I implement the given algorithm and create a function approximator.

In [ ]:
# Approximating the function
def f_approx(x, f, N, M):
    """Approximates function at inteval [-1;1]
    Inputs:
    x (float, [-1,1]): Evaluation point
    f (callable): Function to be approximated
    N (int): Degree of approximation
    M (int, M<N): Number of evaluations of f
    
    Returns:
    f_approx (float): approximation of f(x) """

    # Step 1: Compute optimal evaluation points, z
    def z(k,M):
        return -np.cos(((2*k-1)/(2*M))*np.pi)
    zs = np.array([z(k, M) for k in range(1, M+1)])


    # Step 2: Evaluate function at evaluation points
    ys = np.array([f(z) for z in zs])
    
    # Step 3: Compute approximation coefficients
    def T(i, x):
        return np.cos(i*np.arccos(x))

    f_approx =0
    for i in range(N+1):
        Tzs = np.array([T(i, z) for z in zs])
        a = ((Tzs*ys).sum())/((Tzs**2)).sum()
        f_approx += a*T(i, x)

    # Step 4: Return approximation
    return f_approx

#### Question 2
I test `f_approx` by evaluating it at $x \in \{-0.5, 0.0, 0.98\}$. Approximated values and deviations from the true values are reported below.

In [ ]:
# Setup
f = lambda x: 1/(1+x**2) + x**3 - 0.5*x
N = 5
M = 8
xs = np.array([-0.5, 0.0, 0.98])

# Approximate
approxs = [f_approx(x, f, N, M) for x in xs]
trues = [f(x) for x in xs]
for i in range(len(xs)):
    print(f'Function evaluated at: {xs[i]:6.2f},      Approx. value: {approxs[i]:6.2f},     Deviation: {approxs[i]-trues[i]:6.2f}')